In [1]:
import cot_reports as cot
import pandas as pd
import yfinance as yf


In [5]:
import pandas as pd
import yfinance as yf

def merge_cot_and_price(contract_code, instrument_code, start_year, end_year):
    # COT data
    dfs = []
    for i in range(start_year, end_year + 1):
        single_year = pd.DataFrame(cot.cot_year(i, cot_report_type='legacy_fut'))
        dfs.append(single_year)

    df_cot = pd.concat(dfs)

    asset_cot = df_cot[df_cot['CFTC Contract Market Code (Quotes)'] == contract_code]
    asset_cot = asset_cot[['As of Date in Form YYYY-MM-DD', 'Market and Exchange Names',
                           'Open Interest (All)', 'Noncommercial Positions-Long (All)', 'Noncommercial Positions-Short (All)',
                           'Commercial Positions-Long (All)', 'Commercial Positions-Short (All)',
                           'Nonreportable Positions-Long (All)', 'Nonreportable Positions-Short (All)']].copy()

    # Net Positions
    asset_cot['Net_Position_NonComm'] = asset_cot['Noncommercial Positions-Long (All)'] - asset_cot['Noncommercial Positions-Short (All)']
    asset_cot['Net_Position_Comm'] = asset_cot['Commercial Positions-Long (All)'] - asset_cot['Commercial Positions-Short (All)']
    asset_cot['Net_Position_NonRept'] = asset_cot['Nonreportable Positions-Long (All)'] - asset_cot['Nonreportable Positions-Short (All)']

    # Datetime and index
    asset_cot['As of Date in Form YYYY-MM-DD'] = pd.to_datetime(asset_cot['As of Date in Form YYYY-MM-DD'])
    asset_cot = asset_cot.set_index('As of Date in Form YYYY-MM-DD').sort_index()

    # COT Index for noncom, com, retail and oi
    def calculate_cot_index(series):
        low = series.min()
        high = series.max()
        current_week = series.iloc[-1]

        cot_index = (current_week - low) / (high - low)
        cot_index = round(cot_index * 100, 1)

        return cot_index

    asset_cot['COT_Index_NonComm'] = asset_cot['Net_Position_NonComm'].rolling(26).apply(calculate_cot_index)
    asset_cot['COT_Index_Comm'] = asset_cot['Net_Position_Comm'].rolling(26).apply(calculate_cot_index)
    asset_cot['COT_Index_NonRept'] = asset_cot['Net_Position_NonRept'].rolling(26).apply(calculate_cot_index)
    asset_cot['COT_Index_OI'] = asset_cot['Open Interest (All)'].rolling(26).apply(calculate_cot_index)

    # Price data
    asset_price = yf.download(instrument_code, start=f"{start_year}-01-01", end=f"{end_year}-12-31", progress=False)

    # Merged DataFrame
    merged_df = pd.merge_asof(asset_price, asset_cot, left_index=True, right_index=True, direction='nearest')

    return merged_df


In [6]:

"""
EURO FX - CHICAGO MERCANTILE EXCHANGE                                Code-099741, 6E=F,
BRITISH POUND - CHICAGO MERCANTILE EXCHANGE                          Code-096742, 6B=F,
CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE                        Code-090741
SWISS FRANC - CHICAGO MERCANTILE EXCHANGE                            Code-092741
MEXICAN PESO - CHICAGO MERCANTILE EXCHANGE                           Code-095741
JAPANESE YEN - CHICAGO MERCANTILE EXCHANGE                           Code-097741
EURO FX/BRITISH POUND XRATE - CHICAGO MERCANTILE EXCHANGE            Code-299741
NZ DOLLAR - CHICAGO MERCANTILE EXCHANGE                              Code-112741
BITCOIN - CHICAGO MERCANTILE EXCHANGE                                Code-133741, BTC-USD
MICRO BITCOIN - CHICAGO MERCANTILE EXCHANGE                          Code-133742
E-MINI S&P 500 - CHICAGO MERCANTILE EXCHANGE                         Code-13874A - ES=F
NASDAQ MINI - CHICAGO MERCANTILE EXCHANGE                            Code-209742
AUSTRALIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE                      Code-232741
RUSSELL E-MINI - CHICAGO MERCANTILE EXCHANGE                         Code-239742
BUTTER (CASH SETTLED) - CHICAGO MERCANTILE EXCHANGE                  Code-050642
LEAN HOGS - CHICAGO MERCANTILE EXCHANGE                              Code-054642
LIVE CATTLE - CHICAGO MERCANTILE EXCHANGE                            Code-057642
LUMBER - CHICAGO MERCANTILE EXCHANGE                                 Code-058644

"""

# Usage:
contract_code = '099741'
instrument_code = '6E=F'
start_year = 2022
end_year = 2024

result = merge_cot_and_price(contract_code, instrument_code, start_year, end_year)
result.tail(10)

Selected: legacy_fut
Downloaded single year data from: 2022
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2023
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2024
Stored the file annual.txt in the working directory.
               Open     High      Low    Close  Adj Close  Volume  \
Date                                                                
2022-01-03  1.13790  1.13790  1.12895  1.13005    1.13005     590   
2022-01-04  1.13015  1.13230  1.12765  1.12910    1.12910     408   
2022-01-05  1.12965  1.13455  1.12965  1.13130    1.13130     953   
2022-01-06  1.13190  1.13200  1.12890  1.12890    1.12890     132   
2022-01-07  1.12990  1.13650  1.12950  1.13650    1.13650    1242   
...             ...      ...      ...      ...        ...     ...   
2024-01-23  1.08915  1.09265  1.08340  1.08565    1.08565    1007   
2024-01-24  1.08690  1.09400  1.08645  1

c:\Users\jakub\anaconda3\envs\algovibes\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [ ]:
csv_filename = f"{instrument_code}_{contract_code}_{start_year}_{end_year}_merged.csv"
result.to_csv(csv_filename, index=True)